In [1]:
# Load the drive to access the images and annotations
from google.colab import drive
drive.mount('/content/gdrive')

# reduce the path of the drive
def driveSymboLink():
  # this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
  !ln -s /content/gdrive/My\ Drive/ /mydrive
  # !ls /mydrive

driveSymboLink()

Mounted at /content/gdrive


In [21]:
!pip install opencv-python-headless==4.5.4.60 fiftyone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47.6 MB 1.5 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.2.0.34
    Uninstalling opencv-python-headless-4.2.0.34:
      Successfully uninstalled opencv-python-headless-4.2.0.34


In [22]:
!pip install opencv-python-headless==4.5.4.60

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import torch
import fiftyone as fo
import numpy as np
import os
import shutil, sys

NumExpr defaulting to 2 threads.


In [2]:
print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 1.11.0+cu113 CPU


In [3]:

WDIR='/mydrive/MIDS/W210/Animal_Identification'

In [4]:
def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)


In [7]:
def create_yolov5_dataset1(dataset, train_test_split = 0.8, output_folder='/content/gdrive/MyDrive/MIDS/W210/subset'):
     
    import numpy as np
    import os
    import shutil, sys

    train_files = []
    test_files = []
    np.random.seed(42)
    leopard_dict = {}
    class_count = 0
    train_image_path =  output_folder+'/train'
    isExist = os.path.exists(train_image_path)

    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs(train_image_path+'/images')
        os.makedirs(train_image_path+'/labels')

    test_image_path =  output_folder+'/test'
    isExist = os.path.exists(test_image_path)

    if not isExist:
        os.makedirs(test_image_path+'/images')
        os.makedirs(test_image_path+'/labels')

    for sample in dataset:
      boxes = len(sample['ground_truth']['detections'])
      anno = [[0] * 5 for i in range(boxes)]
      for i in range(boxes):
        anno[i][1:] = sample['ground_truth']['detections'][i]['bounding_box']
        name = sample['ground_truth']['detections'][i]['label']
        if name == 'leopards':
          anno[i][0] = 0
        elif name == 'head':
          anno[i][0] = 1
        else:
          anno[i][0] = 2

        if name not in leopard_dict.keys():
            leopard_dict[name] = 'leop_'+str(class_count)+'_'
            class_count = class_count + 1

        file_suffix = leopard_dict[name]
        height = sample['metadata']['height']
        width = sample['metadata']['width']


        if anno[i][3] >=1.0:
            anno[i][3] = 1-1.0/width

        if anno[i][4] >=1.0:
            anno[i][4] = 1-1.0/height

        anno[i][1] = anno[i][1]+0.5*anno[i][3]
        anno[i][2] = anno[i][2]+0.5*anno[i][4]
      old_image_file = sample['filepath']
      new_file_name =  old_image_file.split("/")[-1][-10:]
      new_file_name = file_suffix + new_file_name
      if(np.random.random_sample() < train_test_split):

          new_image_file = train_image_path + '/images/'+ new_file_name
          new_anno_file = train_image_path + '/labels/'+ new_file_name

      else:
          new_image_file =  test_image_path + '/images/'+ new_file_name
          new_anno_file = test_image_path + '/labels/'+ new_file_name

      new_anno_file = new_anno_file.replace("jpg", "txt")

      copy_str = 'cp ' + old_image_file + ' ' + new_image_file
      os.system(copy_str)

      with open(new_anno_file, 'w') as fp:
          for i in range(boxes):
            for j in range(5):
            # write each item on a new line
              fp.write("%s " % anno[i][j])
            fp.write("\n")
          fp.write("\n")


# create_yolov5_dataset1(dataset1)

In [6]:
# The directory containing the source images
data_path = "/content/gdrive/MyDrive/MIDS/W210/subset/train"

# The path to the COCO labels JSON file
labels_path = "/content/gdrive/MyDrive/MIDS/W210/subset/train/_annotations.coco.json"

# Import the dataset
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=data_path,
    labels_path=labels_path
)

 100% |███████████████| 4946/4946 [14.1s elapsed, 0s remaining, 363.4 samples/s]      


In [15]:
ix=0
for sample in dataset:
  if ix==1:
    break
  else:
    print(sample)
    ix+=1

<Sample: {
    'id': '62c4ff811c475fc2670c4f92',
    'media_type': 'image',
    'filepath': '/content/gdrive/MyDrive/MIDS/W210/subset/train/leop_259_003581_jpg.rf.00004140720a7cee718512d89e0bfeaa.jpg',
    'tags': BaseList([]),
    'metadata': <ImageMetadata: {
        'size_bytes': None,
        'mime_type': None,
        'width': 2400,
        'height': 1800,
        'num_channels': None,
    }>,
    'ground_truth': <Detections: {
        'detections': BaseList([
            <Detection: {
                'id': '62c4ff811c475fc2670c4f91',
                'attributes': BaseDict({}),
                'tags': BaseList([]),
                'label': 'leopard',
                'bounding_box': BaseList([
                    0.47708333333333336,
                    0.6177777777777778,
                    0.34708333333333335,
                    0.33815000000000006,
                ]),
                'mask': None,
                'confidence': None,
                'index': None,
             

In [14]:
create_yolov5_dataset1(dataset)